# Examples for some common ice fabrics

We walk through the matrix model math for some common fabrics. 
- Uniform vertical girdle
- Vertical girdle which has a strengthening magnitude with depth
- Vertical girdles with various rotations (instantaneous and gradual)
- Moveout experiment for a single maximum COF oriented upward

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB

from impdar.lib.ApresData.load_quadpol import load_quadpol_fujita

%matplotlib inline

In [ ]:
### Model functions to be used below

def model(zs,dzs,psis,thetas,chis,
          fc = 300e6, Temp = 253., epsr = 3.15):
    # instantiate the model
    em = effective_medium()
    # system properties
    em.system_setup(fc)
    # material properties
    em.epsr = epsr
    em.ice_properties(T=Temp,epsr=epsr,chi=chis[0])
    # solve the model with a unique case if there is only one layer
    if len(chis) == 1:
        em.solve(zs,dzs,thetas,psis,chis[0])
    else:
        em.solve(zs,dzs,thetas,psis,chis)
    return em

def data(em,n_thetas=201,dz=2.):
    # load the model output as an ImpDAR data object
    dat = load_quadpol_fujita(em)
    # rotate through azimuths
    dat.rotational_transform(n_thetas=n_thetas)
    # find the extinction axis
    dat.find_cpe(Wn=2e7)
    # co-polarization coherence
    dat.coherence2d(.15,dz)
    return dat

def plot(dat,H=1000.,psi_plot=False):
    # 2d grid for all depths and azimuths
    Θs,Ds = np.meshgrid(dat.thetas,dat.range)

    # create a figure
    plt.figure(figsize=(8,4))

    # co-polarization power
    ax1 = plt.subplot(131)
    plt.pcolormesh(Θs,Ds,np.real(dB(dat.HH**2.)),cmap='Greys_r',vmin=-10,vmax=0.)
    plt.ylim(H,0)
    plt.xticks([0,np.pi/2.,np.pi])
    ax1.set_xticklabels(['0','π/2','π'])
    plt.xlabel('Azimuth (rad)')
    plt.ylabel('Depth (m)')
    cbar = plt.colorbar(label='Co-polarization power (dB)',extend='min',location='top',ticks=[-10,-5,0])

    # cross-polarization power and extinction axis
    ax2 = plt.subplot(132,facecolor='k')
    plt.tick_params(labelleft=False)
    plt.pcolormesh(Θs,Ds,np.real(dB(dat.HV**2.)),cmap='Greys_r',vmin=-10,vmax=0.)
    if psi_plot:
        plt.plot(psis,Ds,'lime')
    plt.plot(dat.cpe,Ds,'m.',ms=2)
    plt.ylim(H,0)
    plt.xticks([0,np.pi/2.,np.pi])
    ax2.set_xticklabels(['0','π/2','π'])
    plt.xlabel('Azimuth (rad)')
    cbar = plt.colorbar(label='Cross-polarization power (dB)',extend='min',location='top',ticks=[-10,-5,0])

    # co-polarization phase
    ax3 = plt.subplot(133)
    plt.tick_params(labelleft=False)
    plt.pcolormesh(Θs,Ds,np.angle(dat.chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi)
    plt.ylim(H,0)
    plt.xticks([0,np.pi/2.,np.pi])
    ax3.set_xticklabels(['0','π/2','π'])
    plt.xlabel('Azimuth (rad)')
    cbar = plt.colorbar(label='Phase angle ($\Phi_{HHVV}$)',location='top',ticks=[-np.pi,0,np.pi])
    cbar.ax.set_xticklabels(['-π','0','π'])
    
    plt.tight_layout()

In [ ]:
### Uniform vertical girdle

# geometry
H = 1000.
zs = np.arange(10,H)

# anisotropy strength and orientation
chis = np.array([[.1,0.,0.83]])
thetas = 0.
psis = 0.001

# run model
em = model(zs,H,psis,thetas,chis)
# process output
dat = data(em)
# plot
plot(dat)

## Strengthening Vertical Girdle

In [ ]:
### Strengthening vertical girdle

# geometry
layer_dz = 2.1
zs = np.arange(.1,H+layer_dz,layer_dz)
dzs = layer_dz*np.ones(len(zs))

# anisotropy strength and orientation
thetas = np.zeros(len(zs))
psis = 0.001*np.ones(len(zs))
chi_x = np.linspace(.26,.4,len(zs))
chi_y = np.linspace(.24,.1,len(zs))
chi_z = 0.5*np.ones(len(zs))
chis = np.transpose([chi_x,chi_y,chi_z])

# run model
em = model(zs,dzs,psis,thetas,chis)
# process output
dat = data(em,dz=layer_dz+0.1)
# plot
plot(dat)

In [ ]:
### Rotating vertical girdle (instantaneous rotation of 90 degrees)

# geometry
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
dzs = layer_dz*np.ones(len(zs))

# anisotropy strength and orientation
thetas = np.zeros(len(zs))
psis = 1.*np.pi/4.*np.ones_like(zs)
psis[zs>350] = 3.*np.pi/4.
chis = np.array([.35,.15,0.5])
chis = np.tile(chis,(len(zs),1))

# run model
em = model(zs,dzs,psis,thetas,chis)
# process output
dat = data(em,dz=layer_dz+0.1)
# plot
plot(dat,psi_plot=True)

In [ ]:
### Rotating vertical girdle (gradual rotation)

# geometry
layer_dz = 2.
zs = np.arange(1,H,layer_dz)
dzs = layer_dz*np.ones(len(zs))

# anisotropy strength and orientation
chis = [.35,.15,0.5]
chis = np.tile(chis,(len(zs),1))
thetas = np.zeros(len(zs))
psis = .3*np.tanh(.01*(zs-400))+.5

# run model
em = model(zs,dzs,psis,thetas,chis)
# process output
dat = data(em,dz=layer_dz+0.1)
# plot
plot(dat,psi_plot=True)

In [ ]:
# Moveout experiment for single maximum COF
# This one is slightly different from the above so does not use the functions

# instantiate model
em = effective_medium()

# geometry
H = 1000
layer_dz = 4.
zs = np.arange(1,H,layer_dz)
# antenna separations
seps = np.linspace(0,2000,250)

# system properties (with a placeholder antenna separation)
fc = 300e6
em.system_setup(fc,antenna_sep=seps[0],H=H)

# material properties
Temp = 253.
epsr = 3.15
em.epsr = epsr
chis = np.array([0.3, 0.3, 0.4])
em.ice_properties(T=Temp,epsr=epsr,chi=chis)

# anisotropy strength and orientation
thetas = .01*np.ones(len(zs))
psis = .01*np.ones(len(zs))
dzs = layer_dz*np.ones(len(zs))
chis = np.tile(chis,(len(zs),1))

# prefill output arrays)
HH = np.empty((len(zs),len(seps)),dtype=complex)
VV = np.empty((len(zs),len(seps)),dtype=complex)
HV = np.empty((len(zs),len(seps)),dtype=complex)
# for all the antenna separations
for i in range(len(seps)):
    # print statement to track progress
    if i%10 == 0:
        print(i,'of',len(seps))
    # reset the system properties for new antenna separation
    em.system_setup(fc,antenna_sep=seps[i],H=H)
    # solve the model
    em.solve(zs,dzs,thetas,psis,chis,idctx='uniaxial')
    # save output to arrays
    HH[:,i],VV[:,i],HV[:,i] = em.shh,em.svv,em.shv

# now calculate co-polarized phase coherence of the image
chhvv = np.empty_like(HH)  # empty to fill
for i in range(len(zs)):
    for j in range(len(seps)):
        s1 = HH[i,j]
        s2 = VV[i,j]
        top = np.dot(s1, np.conj(s2))
        bottom = np.sqrt(np.abs(s1)**2.*np.abs(s2)**2.)
        c = top/bottom
        chhvv[i,j] = c

In [ ]:
plt.figure(figsize=(8,4))

ax1 = plt.subplot(131)
HHdb = np.real(dB(HH**2.))
plt.pcolormesh(seps,zs,HHdb,cmap='Greys_r')
plt.ylim(1000,0)
plt.xlabel('Antenna separation (m)')
plt.ylabel('Depth (m)')
cbar = plt.colorbar(label='Co-polarization power (dB)',extend='min',location='top',ticks=[np.min(HHdb),np.max(HHdb)])
cbar.ax.set_xticklabels(['dim','bright'])

ax2 = plt.subplot(132)
plt.tick_params(labelleft=False)
HVdb = np.real(dB(HV**2.))
plt.pcolormesh(seps,zs,HVdb,cmap='Greys_r',vmin=-100,vmax=-60)
plt.ylim(1000,0)
plt.xlabel('Antenna separation (m)')
cbar = plt.colorbar(label='Cross-polarization power (dB)',extend='min',location='top',ticks=[-100,-60])
cbar.ax.set_xticklabels(['dim','bright'])

ax3 = plt.subplot(133)
plt.tick_params(labelleft=False)
plt.pcolormesh(seps,zs,np.angle(chhvv),cmap='twilight_shifted',vmin=-np.pi,vmax=np.pi)
plt.ylim(1000,0)
plt.xlabel('Antenna separation (m)')
cbar = plt.colorbar(label='Phase angle ($\phi_{HHVV}$)',location='top',ticks=[-np.pi,0,np.pi])
cbar.ax.set_xticklabels(['-π','0','π'])

plt.tight_layout()